In [ ]:
from os import makedirs
from os import listdir
from shutil import copyfile
from random import seed, random

In [ ]:
import os
import sys
from matplotlib import pyplot
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd

In [ ]:
train_dir = '/tmp/images/train/'
test_dir = '/tmp/images/test/'

In [ ]:
# Define CNN model to be trained 
model = Sequential()
model.add(Conv2D(32, (4, 4), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(200, 200, 3)))
model.add(MaxPooling2D((3, 3)))
model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))

# Use the number of classes as the number of outputs in the output layer 
model.add(Dense(3, activation='sigmoid'))

In [ ]:
# Compile the model
opt = SGD(lr=0.001, momentum=0.9)
# Use categorical_crossentropy as the loss function
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy']) 

In [ ]:
# Create data generator for reading the image data from directories
datagen = ImageDataGenerator(rescale=1.0/255.0)
# With reading the images as files use categorical as class mode
train_iterator = datagen.flow_from_directory(train_dir, class_mode='categorical', batch_size=64, target_size=(200, 200))
test_iterator = datagen.flow_from_directory(test_dir, class_mode='categorical', batch_size=64, target_size=(200, 200))

In [ ]:
# Create needed directories for this tutorial
makedirs(name='models/', exist_ok=True)
makedirs(name='plots/', exist_ok=True)


In [ ]:
# Train the model and save the training history 
epochs=10
history = model.fit(train_iterator, steps_per_epoch=len(train_iterator), validation_data=test_iterator, validation_steps=len(test_iterator), epochs=epochs)

# Save the trained model to a file
filename = 'basemodel'
model.save('models/' + filename + '.h5')

In [ ]:
## Estimate the model by calculating accuracy with test images
_, accuracy = model.evaluate(test_iterator, steps=len(test_iterator))
print('> Accuracy: %.2f' % (accuracy * 100.0))

## Draw the training curves
# Draw curve for loss 
pyplot.subplot(211)
pyplot.title('Cross Entropy Loss')
pyplot.plot(history.history['loss'], color='blue', label='train')
pyplot.plot(history.history['val_loss'], color='orange', label='test')

# Draw curves for accuracy
pyplot.subplot(212)
pyplot.title('Classification Accuracy')
pyplot.plot(history.history['accuracy'], color='blue', label='train')
pyplot.plot(history.history['val_accuracy'], color='orange', label='test')

# Save plots to one file
pyplot.savefig('plots/' + filename + '_plot.png')
pyplot.close()

In [ ]:
from google.colab import files
files.download('models/basemodel.h5')